In [1]:
from pyspark.sql import SparkSession

MAX_MEMORY = "5g"
spark = SparkSession.builder.master("local").appName("taxi_ml")\
    .config("spark.executer.memory", MAX_MEMORY)\
    .config("spark.drive.memory", MAX_MEMORY).getOrCreate()

23/02/08 18:00:07 WARN Utils: Your hostname, imhaneul-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.27 instead (on interface en0)
23/02/08 18:00:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/08 18:00:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/08 18:00:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
trip_files = "/Users/imhaneul/Documents/sky-laboratory/spark-distribute/data/trips/*"
trip_data = spark.read.parquet(f"file:///{trip_files}")

In [3]:
trip_data.printSchema()
trip_data.createOrReplaceTempView("trips")

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



### 거리당 요금을 예측해보자..!


In [5]:
qs = """
SELECT 
    trip_distance,
    total_amount
FROM
    trips
"""
spark.sql(qs).describe().show()

+-------+-----------------+-----------------+
|summary|    trip_distance|     total_amount|
+-------+-----------------+-----------------+
|  count|         15000936|         15000936|
|   mean|6.628556730060513|  18.755400510292|
| stddev|671.7240645480996|145.7431021908545|
|    min|              0.0|           -647.8|
|    max|        332541.19|         398469.2|
+-------+-----------------+-----------------+



In [34]:
qs = """
SELECT 
    tpep_pickup_datetime,
    total_amount,
    trip_distance
FROM 
    trips
ORDER BY 
    total_amount DESC
"""
spark.sql(qs).show()

+--------------------+------------+-------------+
|tpep_pickup_datetime|total_amount|trip_distance|
+--------------------+------------+-------------+
| 2021-03-18 21:10:41|    398469.2|          0.0|
| 2021-04-10 22:14:49|   395854.74|          5.7|
| 2021-01-05 01:04:51|     7661.28|         2.05|
| 2021-02-05 02:25:03|      6969.3|         2.62|
| 2021-03-28 02:09:56|      6010.8|          1.6|
| 2021-05-03 03:13:44|      5954.0|          0.0|
| 2021-02-11 04:00:54|      4973.3|         5.85|
| 2021-02-07 23:08:10|      3558.0|          0.0|
| 2021-03-30 21:12:07|      2413.8|       964.27|
| 2021-01-20 20:22:05|      2292.4|        427.7|
| 2021-04-29 20:04:15|      2059.3|       821.54|
| 2021-07-27 22:08:58|      1320.8|        25.77|
| 2021-05-27 08:13:15|      1165.3|        16.03|
| 2021-01-12 21:57:36|     1155.65|        326.1|
| 2021-01-03 20:36:52|      1108.2|        267.7|
| 2021-07-31 20:18:09|      988.35|          9.7|
| 2021-06-27 06:31:19|      958.05|          0.0|


In [52]:
qs = """
SELECT 
    total_amount,
    trip_distance
FROM 
    trips
WHERE 
    trip_distance BETWEEN 1 AND 500
    AND total_amount BETWEEN 1 AND 2500 
    AND TO_DATE(tpep_pickup_datetime) < '2022-01-01' 
    AND TO_DATE(tpep_pickup_datetime) < '2022-08-01'
    AND passenger_count > 4
"""
data_df = spark.sql(qs)
data_df.createOrReplaceTempView("data")

In [53]:
data_df.printSchema()

root
 |-- total_amount: double (nullable = true)
 |-- trip_distance: double (nullable = true)



In [54]:
data_df.describe().show()

+-------+------------------+-----------------+
|summary|      total_amount|    trip_distance|
+-------+------------------+-----------------+
|  count|            467036|           467036|
|   mean| 20.12251595163897|3.485057147628969|
| stddev|13.143114935269733|3.765728306936318|
|    min|              1.75|              1.0|
|    max|            561.05|            144.7|
+-------+------------------+-----------------+



In [55]:
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=2023)

In [57]:
from pyspark.ml.feature import VectorAssembler

vectorassemb = VectorAssembler(inputCols=["trip_distance"], outputCol="features")

In [59]:
vtrain_df = vectorassemb.transform(train_df)

In [80]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    maxIter=40,
    labelCol="total_amount",
    featuresCol="features",
    regParam=0.01
)

In [81]:
model = lr.fit(vtrain_df)

In [82]:
vtest_df = vectorassemb.transform(test_df)

In [83]:
pred = model.transform(vtest_df)

In [84]:
print(model.summary.rootMeanSquaredError)
print(model.summary.r2)

4.359258020212368
0.8907464954519646


In [92]:
import seaborn as sns 
import matplotlib.pyplot as plt 

pred_data = pred.show()


+------------+-------------+--------+------------------+
|total_amount|trip_distance|features|        prediction|
+------------+-------------+--------+------------------+
|         5.8|          1.0|   [1.0]|11.925152378056435|
|         6.3|          1.0|   [1.0]|11.925152378056435|
|         6.3|         1.03|  [1.03]|12.024174736540967|
|         6.3|         1.14|  [1.14]|12.387256717650914|
|         6.3|          1.2|   [1.2]|12.585301434619977|
|         6.3|         1.23|  [1.23]|12.684323793104507|
|         6.8|         1.01|  [1.01]|11.958159830884613|
|         6.8|         1.01|  [1.01]|11.958159830884613|
|         6.8|         1.02|  [1.02]| 11.99116728371279|
|         6.8|         1.05|  [1.05]| 12.09018964219732|
|         6.8|         1.08|  [1.08]|12.189212000681852|
|         6.8|         1.09|  [1.09]|12.222219453510029|
|         6.8|          1.1|   [1.1]|12.255226906338207|
|         6.8|          1.1|   [1.1]|12.255226906338207|
|         6.8|          1.1|   